# Сокращённый пайплайн калибровки

In [ ]:
# === 1) Параметры минимального пайплайна ===import os, io, csv, globfrom typing import List, Tuple, Optional, Dictimport numpy as npimport pandas as pdfrom IPython.display import display# Параметры# DATE_FORMAT — формат даты в исходных файлах; оставьте пустым, чтобы pandas подбирал его автоматически.# REF_IDX — индекс эталонного датчика (столбец T{REF_IDX}); относительно него оцениваем остальные каналы.# N_FOLLOW — число соседних значений, которые проверяются вместе при поиске стабильных интервалов.# WINDOW_N — длина скользящего окна в отсчётах для расчёта средних и стандартных отклонений.# STD_THR — максимальное допустимое стандартное отклонение внутри окна; выше — считаем участок нестабильным.# DIFF_THR — допустимая средняя разница между последовательными измерениями; превышение сигналит о скачках.# MIN_LEN — минимальное число точек, чтобы сегмент считался значимым при анализе стабильности.# DEG_TOL — допускаемое расхождение каналов с эталоном в градусах, пока они считаются исправными.# MAX_REF_RANGE — предельный размах значений эталонного канала; если шире, эталон нужно пересмотреть.# GROUP_BY_FILE — при True данные обрабатываются по каждому исходному файлу отдельно, а не одной лентой.DATE_FORMAT = ''REF_IDX = 8N_FOLLOW = 3WINDOW_N = 50STD_THR = 1e-3DIFF_THR = 2e-3MIN_LEN = 20DEG_TOL = 1.0MAX_REF_RANGE = 0.3GROUP_BY_FILE = True# Имена датчиков для отчётаREF_NAME = globals().get('REF_NAME', 'Эталон 2-1')SENSOR_NAMES = globals().get('SENSOR_NAMES', {    'T9': 'Датчик 1-7',    'T10': 'Датчик 1-8',    'T11': 'Датчик 1-9',})# Альтернатива: список имён в порядке датчиков T{REF_IDX+1}..T{REF_IDX+N_FOLLOW}

In [ ]:
# === Загрузка файлов ===import ipywidgets as widgetsfrom IPython.display import displayuploader = widgets.FileUpload(description='Загрузить файлы', accept='.csv,.txt,.zip', multiple=True, layout=widgets.Layout(width='50%'))display(uploader)

In [ ]:
# === Оркестратор пайплайна ===from pathlib import Pathfrom IPython.display import HTMLimport json as _jsonuploads_dir = Path('uploads')if 'uploader' in globals() and getattr(uploader, 'value', None):    uploads_dir.mkdir(exist_ok=True)    value = uploader.value.values() if isinstance(uploader.value, dict) else uploader.value    saved = 0    for item in value:        data = item.get('content')        name = item.get('name') or item.get('metadata', {}).get('name')        if name and data is not None:            (uploads_dir / name).write_bytes(data)            saved += 1    print(f'Загружено файлов: {saved}')else:    print('Новых файлов не загружено — будут использованы локальные файлы проекта.')nb_json = _json.loads(Path('temperature_stability_modular_v2.ipynb').read_text(encoding='utf-8'))first_code = Truefor cell in nb_json['cells']:    if cell.get('cell_type') != 'code':        continue    src = ''.join(cell.get('source', []))    if first_code:        first_code = False        continue    exec(compile(src, 'temperature_stability_modular_v2.ipynb', 'exec'), globals())print('Пайплайн выполнен.')try:    display(HTML('<a href="calibration_report.html" target="_blank">Открыть отчёт</a>'))except Exception:    pass